<a href="https://colab.research.google.com/github/nihalbaig0/dphi_notebooks/blob/master/banknote.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
from keras.utils.np_utils import to_categorical
%load_ext tensorboard

# Helper libraries
import numpy as np
import random
import matplotlib.pyplot as plt
import datetime

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [15]:
import pandas as pd

bank_note_data = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/bank_note_data/training_set_label.csv" )

In [16]:
bank_note_data.head()


,VWTI,SWTI,CWTI,EI,Class
0,2.2634,-4.4862,3.6558,-0.612510,0
1,3.2718,1.7837,2.1161,0.613340,0
2,-3.9411,-12.8792,13.0597,-3.312500,1
3,0.5195,-3.2633,3.0895,-0.984900,0
4,2.5698,-4.4076,5.9856,0.078002,0


In [17]:
bank_note_data.describe()

,VWTI,SWTI,CWTI,EI,Class
count,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000
mean,0.448500,1.780643,1.493533,-1.157454,0.445255
std,2.852623,5.922621,4.375655,2.084983,0.497221
min,-7.036400,-13.773100,-5.286100,-8.548200,0.000000
25%,-1.790850,-2.125200,-1.574975,-2.246975,0.000000
50%,0.540430,2.205850,0.671900,-0.569190,0.000000
75%,2.835350,6.793925,3.574450,0.399980,1.000000
max,6.563300,12.730200,17.927400,2.449500,1.000000


In [18]:
#set the seed
from numpy.random import seed
seed(1)

In [19]:
bank_note_data.shape

(1096, 5)

In [40]:
predictors = bank_note_data.drop(['Class'],axis=1).to_numpy()
target = to_categorical(bank_note_data.Class)

In [41]:
predictors.shape[1]


4

In [46]:
def create_model():
  model = Sequential()
  #input layer size is 784 after flattening
  model.add(Dense(100,activation='relu',input_shape=(predictors.shape[1],)))
  #hidden layer with 512 neurons
  model.add(Dense(512, activation='relu'))
  model.add(Dense(10, activation='softmax'))
  return model

In [47]:
model = create_model()
model.summary()
model.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
earlystopping_callback = EarlyStopping(
    monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto',
    baseline=None, restore_best_weights=True
)
#modelcheckpoint_callback = ModelCheckPoint('best_model.hdf5',monitor='val_loss', verbose=0, mode='auto',save_best_only=True)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 100)               500       
_________________________________________________________________
dense_7 (Dense)              (None, 512)               51712     
_________________________________________________________________
dense_8 (Dense)              (None, 10)                5130      
Total params: 57,342
Trainable params: 57,342
Non-trainable params: 0
_________________________________________________________________


In [49]:
model.fit(predictors, 
          target, 
          epochs=20, 
          validation_split=0.2, 
          callbacks=[earlystopping_callback])

Epoch 1/20


InvalidArgumentError: ignored